In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

from statsmodels.stats.multicomp import pairwise_tukeyhsd

import re
import nltk


from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [37]:
def clear():
    os.system('cls')

def ListNoDups(mylist):
    mylist = list(dict.fromkeys(mylist))
    return mylist

class CarData:
    
    missing = -1
    duplicates = -1
    
    price_outliers = -1
    mileage_outliers = -1
    year_outliers = -1
    total_discard = -1
    
    corpus = []
    def __init__(self, path, price_outlier_mt = 200000, mileage_outlier_mt = 400000,
                year_outlier_lt = 1995, engine_outlier_mt = 4000, engine_outlier_lt = 750,
                dependent_variable = 'price',
                categorical_variables = ['engine_type', 'city', 'province'],
                numeric_variables = ['price', 'mileage_km', 'engine_cm3', 'year']):
        
        #define outliers values
        self.price_outlier_mt = price_outlier_mt
        self.mileage_outlier_mt = mileage_outlier_mt
        self.year_outlier_lt =  year_outlier_lt
        self.engine_outlier_mt = engine_outlier_mt
        self.engine_outlier_lt = engine_outlier_lt
        self.dependent_variable = dependent_variable
        

        
        
        #define variable data types
        self.numeric_variables = numeric_variables
        self.categorical_variables = categorical_variables
        
        #read all .csv files from the directory
        self.data = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "*.csv"))), sort=False)
        
        #drop the duplicates and save the number of duplicates - many duplicates due to data gathering method
        self.duplicates = len(self.data) - len(self.data.drop_duplicates())
        self.data.drop_duplicates(inplace = True)
        
        #rename columns
        self.data.columns = ['title', 'price', 'sub_title', 'mileage_km', 'year', 'engine_cm3',
                'engine_type', 'city', 'province', 'negotiable']
        
        #drop NaNs and save the number of rows dropped to the missing varaible
        self.missing = self.data['engine_type'].isna().sum()
        self.data.dropna(subset = ['engine_type'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['city'].isna().sum()
        self.data.dropna(subset = ['city'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['engine_cm3'].isna().sum()
        self.data.dropna(subset = ['engine_cm3'], axis = 'index', inplace = True)
        
        #clean up the columns
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(",", ""))
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(" ", "")).astype('int')
        
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace("km", ""))
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace(" ", "")).astype('float')
        
        self.data['engine_cm3'] = self.data['engine_cm3'].astype('str')
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace('cm3', ''))
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace(' ','')).astype('int')
        
        self.data['province'] = self.data['province'].astype('str')
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('(',''))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace(')',''))
        
        self.data['sub_title'] = self.data['sub_title'].astype('str') #may change that in the future - possible info loss due to lowercase
        
        
        self.data['title'] = self.data['title'].astype('str') #may change that in the future - possible info loss due to lowercase
        
        self.data['negotiable'] = self.data['negotiable'].astype('str')
        
        #Add ID column
        self.data.insert(loc = 0, column = 'ID', value = range(1, len(self.data)+1))

        #discard outliers and calculate the numbers
        self.total_discard = len(self.data) - len(self.data[(self.data['price'] <= self.price_outlier_mt) &
                                                        (self.data['mileage_km'] <= self.mileage_outlier_mt) &
                                                        (self.data['year'] >= self.year_outlier_lt) &
                                                        (self.data['engine_cm3'] <= self.engine_outlier_mt) &
                                                        (self.data['engine_cm3'] >= self.engine_outlier_lt)])        
        
        self.price_outliers = len(self.data[self.data['price'] > price_outlier_mt])
        self.data = self.data[self.data['price'] <= price_outlier_mt]
        
        self.mileage_outliers = len(self.data[self.data['mileage_km'] > mileage_outlier_mt])
        self.data = self.data[self.data['mileage_km'] <= mileage_outlier_mt]
        
        self.year_outliers = len(self.data[self.data['year'] < year_outlier_lt])
        self.data = self.data[self.data['year'] >= year_outlier_lt]
        
        self.engine_outliers = len(self.data[(self.data['engine_cm3'] > engine_outlier_mt) |
                                            (self.data['engine_cm3'] < engine_outlier_lt)])
        self.data = self.data[(self.data['engine_cm3'] <= engine_outlier_mt) & 
                             (self.data['engine_cm3'] >= engine_outlier_lt)]
        
        #NLP
        self.data['concat_title_subtitle'] = self.data['title'] + ' ' + self.data['sub_title']
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.lower())
        
        #replace problematic cases for NLP for title and subtitle
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('+',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('(',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(')',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('**',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('*',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(']',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('[',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace("/"," "))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace("\\"," "))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(',',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('?',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('.',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('!',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('_',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('-',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('|',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('#',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('%',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('~',' '))  
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('*',' ')) 
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('*',' '))
        
        #NLP preprocessing for location
        self.data['province'] = self.data['province'].apply(lambda x: x.lower()) 
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ą', 'a')) 
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ę', 'e'))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ł', 'l'))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ś', 's'))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ć', 'c'))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('ż', 'z'))
        
    def describe(self):
        #descriptive statistice
        desc_stats = round(pd.DataFrame(
                        data = self.data[self.numerical_variables].describe(),
                        columns = self.data[self.numerical_variables].columns),2)
        return desc_stats
    
    def outliers(self):
        #baisc data about outliers discarded during preprocessing
        print('Offers with price greater than '+str(self.price_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.price_outliers))
        print('')
        print('Offers with mileage greater than '+str(self.mileage_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.mileage_outliers))
        print('')
        print('Offers with year lower than '+str(self.year_outlier_lt)+' have been discarded')
        print('The number of such offers = '+str(self.year_outliers))
        print('')
        print('Offers with engine_cm3 greater than '+str(self.engine_outlier_lt)+
              ' and lower than '+str(self.engine_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.engine_outliers))
        print('')
        print('Total number of discarded offers = '+str(self.total_discard)
              +'('+str(round(self.total_discard/len(self.data)*100,2))+'%)'
              +' - may be different to the sum of above due to overlap')
        
    def scatter_nox(self, var = 'all', figsize_1 = 7, figsize_2 = 5):
        #prints scatter plots with no x axis - a dummy sequence as x axis
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        if var == 'all':
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data['mileage_km'], x = range(1, len(self.data)+1), s=1)
            plt.title('mileage_km')
            plt.show()

            plt.scatter(y = self.data['price'], x = range(1, len(self.data)+1), s=1)
            plt.title('price')
            plt.show()

            plt.scatter(y = self.data['year'], x = range(1, len(self.data)+1), s=1)
            plt.title('year')
            plt.show()

            plt.scatter(y = self.data['engine_cm3'], x = range(1, len(self.data)+1), s=1)
            plt.title('engine_cm3')
            plt.show()
        else:
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data[var], x = range(1, len(self.data)+1), s=1)
            plt.title(var)
            plt.show()
            
    def scatter(self, var = 'all'):
        #prints scatter plots for numerical variables
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        list_comb = []
        if var == 'all':
            for variable1 in enumerate(self.numeric_variables):
                for variable2 in enumerate(self.numeric_variables):
                    if variable1 != variable2 and variable1[1]+variable2[1] not in list_comb and variable2[1]+variable1[1] not in list_comb:
                        plt.scatter(y = self.data[variable1[1]], x = self.data[variable2[1]], s=1)
                        plt.title("Correlation between "+variable1[1]+' and '+variable2[1])
                        plt.ylabel(variable1[1])
                        plt.xlabel(variable2[1])
                        plt.show()
                        list_comb.append(variable1[1]+variable2[1])
        #else: - TO DO
    
    def hist(self, var = 'all', bins = 50):
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        #prints histograms for numeric variables
        if var == 'all':
            for variable in enumerate(self.numeric_variables):
                plt.hist(x = self.data[variable[1]], bins = bins)
                plt.title(variable[1])
                plt.show()
     
    def price_cat_vars(self, variables = '_NULL_'):
        
        if variables == '_NULL_':
            variables = self.categorical_variables
            
        for variable in enumerate(variables):
            # shows desrptive statistics of categorical variables
            print(self.data.groupby(self.data[variable[1]])['price'].describe())
            #the variables need further preprocessing
            
    def add_dummies(self, categorical_list, columns_to_check, delete_from_strings = 'yes'):
        #adds dummmies from cat_list, checks in every column of columns_to_check
        for column in enumerate(columns_to_check):
            for category in enumerate(categorical_list):
                col_name = column[1] + '_' + category[1]
                self.data[col_name] = self.data[column[1]].str.contains(category[1]).astype('int')
                
                #append newly craeted varaibles to categorical variables
                if self.data[col_name].sum() > 0:
                    self.categorical_variables.append(col_name)
                else:
                    self.data.drop(columns = [col_name], inplace = True)
                
                #delete the string from the column
                if delete_from_strings == 'yes':
                    self.data[column[1]] = self.data[column[1]].apply(lambda x: x.replace(category[1], ''))
                
        return self
    
    def add_dummies2(self, categorical_list, delete_from_strings = 'yes'):
        #adds dummmies from cat_list, checks in concat_title_subtitle column
        for category in enumerate(categorical_list):
            
            #print(category)
            
            col_name = category[1]
            self.data[col_name] = self.data['concat_title_subtitle'].str.contains(category[1]).astype('int')
                
                #append newly craeted varaibles to categorical variables
            if self.data[col_name].sum() > 0:
                self.categorical_variables.append(col_name)
            else:
                self.data.drop(columns = [col_name], inplace = True)
                
                #delete the string from the column
            if delete_from_strings == 'yes':
                self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(category[1], ''))
                
        return self
    """
    def ind_test(self, var, alpha = 0.05):
        if alpha > 1 or alpha < 0:
            print('Incorrect alpha value. Select a value from <0;1>.')
            
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        pivot = round(self.data.pivot_table(values = 'price', index = var, aggfunc = ['count', 'mean']),2)
        pivot.columns = ['count', 'mean']
        
        mean_price = self.data['price'].mean()
        
        pivot['sm'] = pivot['mean']/((pivot['count'])**(1/2))
        
        pivot['t'] = (pivot['mean']-mean_price)/pivot['sm']
        pivot['df'] = pivot['count']-1

        #calculate p-value
        pivot['t_border'] = stats.t.ppf(1-alpha/2, pivot['df'])
        
        #implementation here is not 100% mathematically correct
        return pivot
    """
    def anova(self, var = 'all', alpha = 0.05):
        if var == 'all':
            for variable in enumerate(self.categorical_variables):
                anova_data = self.data[[variable[1], 'price']].reset_index().copy()
                anova_data.columns = ['index', variable[1], 'price']
                equation_string = 'price ~ '+str(variable[1])
                model = ols(equation_string, data=anova_data).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)
                print(anova_table)
                print()
                
                #pairwise comparisons
                pairwise_comparison = pairwise_tukeyhsd(endog = anova_data['price'],
                                                        groups = anova_data[variable[1]],
                                                        alpha = alpha)
                print(pairwise_comparison)
                print()
        #else:
            #TO DO
            #also TO DO check Anova assumptions
    def make_corpus(self):
        #creates a corpus out of title and subtitle column
        for i in range(0, len(self.data)):
            #string = re.sub('[^a-zA-Z]', ' ', self.data.reset_index().loc[i]['concat_title_subtitle'])
            string = self.data.reset_index().loc[i]['concat_title_subtitle']
            string = string.split()
            self.corpus = self.corpus + string
        self.corpus = ListNoDups(self.corpus)
        
        #with open("corpus.txt", "w") as output:
         #   output.write(str(self.corpus))
    
        return self.corpus
    
    def analyse_variables(self, list_of_variables, discard = 0.01):
    # independence tests for a list of variable e.g. corpus
        final_df = pd.DataFrame(columns = ['variable', 'mean_1', 'mean_0', 'count_1', 'count_0'])

        for variable in enumerate(list_of_variables):
            #debug
            print(str(variable)+' done')
            
            self.data[variable[1]] = self.data['concat_title_subtitle'].str.contains(variable[1]).astype('int')

            mean_1 = self.data.loc[self.data[variable[1]] == 1][self.dependent_variable].mean()
            mean_0 = self.data.loc[self.data[variable[1]] == 0][self.dependent_variable].mean()

            count_1 = len(self.data.loc[self.data[variable[1]] == 1])
            count_0 = len(self.data.loc[self.data[variable[1]] == 0])        
            
            if count_1 >= discard * len(self.data) and count_0 >= discard * len(self.data):            
                dict_to_append = {
                    'variable' : variable[1],
                    'mean_1' : mean_1,
                    'mean_0' : mean_0,
                    'count_1' : count_1,
                    'count_0' : count_0
                }

                final_df = final_df.append(dict_to_append, ignore_index = True)

            self.data.drop(columns = [variable[1]], inplace = True)

            #if variable[0] % 1000 == 0:
            #    print(str(variable[0])+'/'+str(len(list_of_variables)))
            
            
        
        
        final_df['mean_diff'] = abs(final_df['mean_1'] - final_df['mean_0'])
        final_df = final_df.sort_values(by = 'mean_diff', ascending = False).reset_index()
        final_df.drop(columns = 'index', inplace = True)
        
        final_df.to_csv('analyse_variables_results.csv')
        
        return final_df

        
            

#cv = CountVectorizer(max_features = 1000)
#X = cv.fit_transform(corpus).toarray()

In [46]:
x = CarData(path = 'data/', dependent_variable = 'price', categorical_variables = ['engine_type', 'city', 'province'],
           numeric_variables = ['price', 'mileage_km', 'engine_cm3', 'year'])

In [6]:
brands = ['alfa', 'audi', 'bmw', 'chevrolet', 'chrysler',
          'citroen', 'dacia', 'daewoo', 'dodge', 'fiat',
          'ford', 'honda', 'hyundai', 'jaguar', 'jeep',
         'kia', 'rover', 'lexus', 'mazda', 'mercedes',
         'mitsubishi', 'nissan', 'opel', 'peugeot',
         'porsche', 'renault', 'seat', 'smart', 'subaru', 
          'suzuki', 'tesla', 'toyota', 'volkswagen',
          'volvo', 'skoda']

In [47]:
x.add_dummies2(brands)

In [8]:
corpus = x.make_corpus()

In [9]:
analysis = x.analyse_variables(corpus, discard = 0.01)

(0, 'cx') done
(1, '3') done
(2, '150km') done
(3, '6at') done
(4, '4x4') done
(5, 'skypassion') done
(6, 'biala') done
(7, 'skóra') done
(8, 'safety') done
(9, 'navi') done
(10, 'partner') done
(11, 'opłacony') done
(12, 'dokumentacja') done
(13, 'przebiegu') done
(14, 'sportage') done
(15, 'opłacona') done
(16, 'napęd') done
(17, 'škoda') done
(18, 'roomster') done
(19, 'serwis') done
(20, 'aso') done
(21, 'klimatronic') done
(22, 'grzane') done
(23, 'siedzenia') done
(24, 'combo') done
(25, 'klimatyzacja') done
(26, 'przebieg') done
(27, 'faktura') done
(28, 'polski') done
(29, 'salon') done
(30, 'lancer') done
(31, '1') done
(32, '8') done
(33, 'polska') done
(34, '24000') done
(35, 'seria') done
(36, '320d') done
(37, '5') done
(38, 'e60') done
(39, '535d') done
(40, '272km') done
(41, '2006r') done
(42, 'head') done
(43, 'up') done
(44, 'start') done
(45, 'stop') done
(46, 'xenony') done
(47, 'bogate') done
(48, 'wyposażenie') done
(49, 'grand') done
(50, 'cherokee') done
(51, 'n

(414, 'el') done
(415, 'pamięć') done
(416, '19"') done
(417, 'drzwi') done
(418, 'śliczna') done
(419, '85km') done
(420, 'ładny') done
(421, 'leszno') done
(422, 'c30') done
(423, '6d') done
(424, '2007r') done
(425, 'polskory') done
(426, 'climatronic') done
(427, 'nowe') done
(428, 'czysty') done
(429, 'zadbany') done
(430, 'x1') done
(431, 'xdrive25i') done
(432, '25i') done
(433, '28i') done
(434, '218d') done
(435, 'gran') done
(436, 'advantage') done
(437, 'golf') done
(438, 'vii') done
(439, '0tdi') done
(440, 'comfortline') done
(441, 'po') done
(442, 'wym') done
(443, 'rozrządu') done
(444, 'wł') done
(445, 'q7') done
(446, 'prywatne') done
(447, 'serwisowane') done
(448, 'krajowe') done
(449, 'casa') done
(450, '600') done
(451, 'classic') done
(452, 'seicento') done
(453, 'van') done
(454, 'najnowszy') done
(455, 'model') done
(456, '15800') done
(457, 'fabia') done
(458, 'klimatyzacją') done
(459, 'ważne') done
(460, '143') done
(461, 'biegów') done
(462, 'a4b7') done
(46

(816, 'tysięcy') done
(817, 'temu') done
(818, 'wy') done
(819, '2003') done
(820, '04') done
(821, 'trend') done
(822, 'do') done
(823, 'końca') done
(824, 'silnikiem') done
(825, 'asx') done
(826, '2018r') done
(827, '12') done
(828, 'e83') done
(829, 'gtc') done
(830, 'coupé') done
(831, 'środek') done
(832, 'grip') done
(833, 'control') done
(834, '39tys') done
(835, 'fluence') done
(836, 'fluensce') done
(837, 'pierwsza') done
(838, 'rejestracja') done
(839, 'type') done
(840, '2009') done
(841, 'kraju') done
(842, 'no') done
(843, '370') done
(844, 'nismo') done
(845, 'vitara') done
(846, 'reduktor') done
(847, 'sprawna') done
(848, 'genesis') done
(849, 'ultimate') done
(850, 'najbogatsza') done
(851, 'czujniki') done
(852, 'brembo') done
(853, 's80') done
(854, '5t') done
(855, '4hdi') done
(856, 'captur') done
(857, 'tce') done
(858, '23800') done
(859, 'igła') done
(860, '118ps') done
(861, 'felicia') done
(862, 'felcia') done
(863, 'glx') done
(864, 'salonowa') done
(865, 'i

(1207, 'nowości') done
(1208, 'b7') done
(1209, 'l') done
(1210, 'parktronik') done
(1211, 'chrom') done
(1212, '35') done
(1213, 'megan') done
(1214, '231') done
(1215, 'ecoblue') done
(1216, 'gwarancji') done
(1217, '2004') done
(1218, 'bęzyna') done
(1219, 'romeo') done
(1220, 'mito') done
(1221, 'zafira') done
(1222, 'osobowa') done
(1223, 'bezwypadkowa') done
(1224, 'krajowa') done
(1225, 'właściciela') done
(1226, 'hdi;') done
(1227, 'tanio') done
(1228, 'enjoy') done
(1229, 'cx3') done
(1230, 'kanada') done
(1231, 'unikat') done
(1232, 'tiv') done
(1233, '2020') done
(1234, 'ubezpieczony') done
(1235, 'navara') done
(1236, 'kolor') done
(1237, 'pieprz') done
(1238, 'foteli') done
(1239, 'alufelga') done
(1240, '16') done
(1241, 'orginale') done
(1242, '147') done
(1243, '2001') done
(1244, 'crafter') done
(1245, 'brygadówka') done
(1246, 'off') done
(1247, 'road') done
(1248, '83km') done
(1249, 'clasic') done
(1250, 'ladny') done
(1251, '2kpl') done
(1252, 'kol') done
(1253, 'o

(1585, 'cali') done
(1586, 'panoramiczny') done
(1587, '190km') done
(1588, 'jeżdżący') done
(1589, 'wj') done
(1590, '4turbo') done
(1591, '2011r') done
(1592, 'toyote') done
(1593, 'mustang') done
(1594, 'discovery') done
(1595, '177km') done
(1596, 'ze') done
(1597, 'szwajcarii') done
(1598, 'grandis') done
(1599, 'ka') done
(1600, 'elektryczny') done
(1601, 'w211') done
(1602, '177') done
(1603, '400') done
(1604, 'nm') done
(1605, 'primera') done
(1606, '6multijet') done
(1607, 'długi') done
(1608, 'maxi') done
(1609, '5os') done
(1610, 'przes') done
(1611, 'iwl') done
(1612, 'srebny') done
(1613, '25800') done
(1614, 'sorento') done
(1615, 'slink') done
(1616, 'trail') done
(1617, '8n') done
(1618, '180hp') done
(1619, 'ml') done
(1620, 'ben') done
(1621, 'remoncie') done
(1622, '55kw') done
(1623, 'one') done
(1624, 'salt') done
(1625, '318d') done
(1626, 'sprzęgło') done
(1627, 'pajero') done
(1628, 'did') done
(1629, '8l') done
(1630, 'par') done
(1631, '2014r') done
(1632, '2

...

(12392, 'sportiv') done
(12393, "my'18") done
(12394, 'boleno') done
(12395, 'nowością') done
(12396, '1298cm') done
(12397, 'renulat') done
(12398, '46400') done
(12399, 'zaworowa') done
(12400, 'climat') done
(12401, 'baaardzo') done
(12402, 'philips') done
(12403, 'fullll') done
(12404, 'około') done
(12405, '380nm') done
(12406, 'mobilo') done
(12407, 'vipgwarant') done
(12408, '1240cm3') done
(12409, 'flexi') done
(12410, 'cleandiesel') done
(12411, '191km') done
(12412, '2di') done
(12413, 'b170') done
(12414, '271000') done
(12415, 'dlugi') done
(12416, '18500') done
(12417, '3xm') done
(12418, 'dowod') done
(12419, 'prezentuje') done
(12420, 'pilotem') done
(12421, '89tyskm') done
(12422, 'edit') done
(12423, 'impuls') done
(12424, 'pełenserwis') done
(12425, 'stag300') done
(12426, 'cmr') done
(12427, '25cm3') done
(12428, 'zbliżeniowy') done
(12429, 'rej2018') done
(12430, 'kutryny') done
(12431, 'fo') done
(12432, 'superwagen') done
(12433, 'kompety') done
(12434, 'zamina') 

(12736, '35"') done
(12737, 'lovells') done
(12738, 'rev') done
(12739, '303ps') done
(12740, 'xz') done
(12741, 'z350') done
(12742, 'z370') done
(12743, '999cm') done
(12744, 'bezwypadkowt') done
(12745, 'rossa') done
(12746, '2030r') done
(12747, 'pasaat') done
(12748, '125oootkm') done
(12749, 'kolekcjonerskim') done
(12750, '19800km') done
(12751, 'pelni') done
(12752, '"x') done
(12753, 'gear') done
(12754, 'delica') done
(12755, '33;') done
(12756, 'wd8806e') done
(12757, 'gd269jf') done
(12758, 'lapmy') done
(12759, '1włas') done
(12760, 'montegoblue') done
(12761, '201000') done
(12762, '3szt') done
(12763, 'sezony') done
(12764, '35400') done
(12765, 'neony') done
(12766, '81600km') done
(12767, 'ga355as') done
(12768, 'cb284hp') done
(12769, 'intenseplus') done
(12770, 'premiumo') done
(12771, '0ide') done
(12772, 'przyjedz') done
(12773, "alu17'") done
(12774, 'moro') done
(12775, '4xes') done
(12776, '154tyś') done
(12777, 'włas') done
(12778, 'felicja') done
(12779, '::za

(13081, 'ltd') done
(13082, '4xclimatronik') done
(13083, 'podgzewana') done
(13084, 'do2018r') done
(13085, 'udukomentowane') done
(13086, 'wd1201m') done
(13087, 'wd9531j') done
(13088, 'wd0223j') done
(13089, 'rivoli') done
(13090, 'ledvision') done
(13091, 'hiundai') done
(13092, '40netto') done
(13093, 'innovision') done
(13094, 'termin') done
(13095, 'powerxtra') done
(13096, '561zł') done
(13097, 'wd7454k') done
(13098, 'wd0943l') done
(13099, 'zs763jx') done
(13100, 'wd0826j') done
(13101, 'wd9985l') done
(13102, 'wd9041l') done
(13103, 'gd435mt') done
(13104, 'tempoat') done
(13105, 'extreme') done
(13106, '120000km') done
(13107, 'd4d143') done
(13108, 'kr5x141') done
(13109, 'wd9984l') done
(13110, 'el8g194') done
(13111, 'wd0836j') done
(13112, 'wd9968l') done
(13113, 'przyjedziesz') done
(13114, 'kupisz') done
(13115, 'gle350') done
(13116, '1sz') done
(13117, '795km') done
(13118, 'kwota') done
(13119, 'bhdi') done
(13120, 'holowanie') done
(13121, '22tys') done
(13122, '

(13422, 'ed30') done
(13423, 'myway') done
(13424, 'comfor') done
(13425, 'krajowej') done
(13426, 'dealerskiej') done
(13427, 'zaprraszam') done
(13428, 'ahh') done
(13429, 'pompie') done
(13430, 'poliftowego') done
(13431, 'uderzona') done
(13432, 'nietuzinkowe') done
(13433, 'chlodnice') done
(13434, 'mjd') done
(13435, 'luxurypack') done
(13436, 'ravat23') done
(13437, '0hdi241ps') done
(13438, 'exclusiveplus') done
(13439, 'obejrzec') done
(13440, 'wpomaganie') done
(13441, '187ps') done
(13442, 'join') done
(13443, 'asysten') done
(13444, 'manuala') done
(13445, '794') done
(13446, 'cla180') done
(13447, '44kw') done
(13448, '52tys') done
(13449, 'afs') done
(13450, "alu17''aso") done
(13451, '83300km') done
(13452, 'finansoanie') done
(13453, 'tourana') done
(13454, 'delikatnieuszk') done
(13455, 'zarejestrpl') done
(13456, '795') done
(13457, 'autostradowy') done
(13458, '5td5') done
(13459, 'toyo') done
(13460, 'j24b') done
(13461, 'dostawadodomu') done
(13462, '153000km') don

(13764, 'tucosn') done
(13765, 'selectione') done
(13766, '114d') done
(13767, '105km;') done
(13768, 'sport;') done
(13769, 'klima;') done
(13770, 'zadbana;') done
(13771, 'zgorzelec') done
(13772, 'autko:') done
(13773, 'm550') done
(13774, '79900mil') done
(13775, 'japonska') done
(13776, 'roweru') done
(13777, 'okazia') done
(13778, 'bwypadkowy') done


In [ ]:
analysis.to_csv('analyse_variables_results.csv')

In [3]:
analysis = pd.read_csv('analyse_variables_results.csv')

In [4]:
analysis.drop(columns = 'Unnamed: 0', inplace = True)

In [10]:
variables = analysis['variable'].values.tolist()

In [11]:
cleaned_variables = [x for x in variables if str(x) != 'nan' and len(x) >=3 ] #let's remove everything shorter than 3 characters

In [48]:
x.add_dummies2(cleaned_variables)

In [13]:
x.data.to_csv('training_set_21-06-2020.csv')

In [14]:
x.data

,ID,title,price,sub_title,mileage_km,engine_cm3,engine_type,province,negotiable,concat_title_subtitle,...,szy,ele,van,ass,grz,nisk,bez,kier,ani,and
0,1,Mazda CX-3,69700,150KM 6AT 4x4 SkyPASSION (+ Biala skóra + Safe...,51015.0,1998,Benzyna,Śląskie,NaN,cx 3 1 6at skysion biala s safety i,...,0,0,0,0,0,0,0,0,0,0
1,2,Peugeot Partner,19500,Opłacony~Dokumentacja Przebiegu,180000.0,1600,Diesel,Mazowieckie,Do negocjacji,tner ay dokumentacja eu,...,0,0,0,0,0,0,0,0,0,0
2,3,Kia Sportage,26800,Opłacona~Napęd 4x4,165000.0,2000,Benzyna,Mazowieckie,Do negocjacji,age aa napęd,...,0,0,0,0,0,0,0,0,0,0
3,4,Škoda Roomster,12900,Opłacona~Serwis ASO~Klimatronic~Grzane siedzenia,238000.0,1600,Benzyna,Mazowieckie,Do negocjacji,ša roomster aa wis tro ne siedzenia,...,0,0,0,0,0,0,0,0,0,0
4,5,Opel Combo,14500,Klimatyzacja~Przebieg~Faktura~Polski Salon,136000.0,1400,Benzyna,Mazowieckie,"Do negocjacji, Faktura VAT",combo e fura,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4827,62074,Ford Focus 1.6,23000,Ford Focus,107000.0,1560,Diesel,Łódzkie,Do negocjacji,1 6,...,0,0,0,0,0,0,0,0,0,0
4828,62075,Mini Cooper S,22500,nan,135340.0,2000,Diesel,Wielkopolskie,Do negocjacji,mini cooper s nan,...,0,0,0,0,0,0,0,0,0,0
4829,62076,Toyota Corolla 1.4,8900,Toyota Corolla 1.4,157500.0,1398,Benzyna,Dolnośląskie,Do negocjacji,clla 1 4 clla 1 4,...,0,0,0,0,0,0,0,0,0,0
4830,62077,Toyota Corolla Verso,12900,Toyota Corolla Verso,206000.0,1995,Diesel,Małopolskie,Do negocjacji,clla verso clla verso,...,0,0,0,0,0,0,0,0,0,0


In [49]:
y = x.data['price'].copy()

In [50]:
x.add_dummies(x.data['engine_type'].unique().tolist(), ['engine_type'])

In [51]:
x.add_dummies(x.data['province'].unique().tolist(), ['province'])

In [56]:
x.data.head()

,ID,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable,concat_title_subtitle,alfa,audi,bmw,chevrolet,chrysler,citroen,dacia,daewoo,dodge,fiat,ford,honda,hyundai,jaguar,jeep,kia,rover,lexus,mazda,mercedes,mitsubishi,nissan,opel,peugeot,porsche,renault,seat,smart,subaru,suzuki,tesla,toyota,volkswagen,volvo,skoda,drive,fv23,v23,20d,pakiet,akiet,pak,quattro,vat,dsg,quat,fort,line,for,akt,kam,amer,fak,led,tsi,pre,klimatyzacja,ful,salon,180,sal,tyl,corsa,polo,klima,klim,raw,aso,tiv,16v,elektr,drzw,ima,drz,gaz,fabia,opłat,elekt,lim,sport,elek,50km,0td,niemie,asa,sprowadzon,spro,wad,niem,150k,lpg,clio,fiesta,fiest,dobry,4x4,ład,opł,201,zadbana,200,polska,alufelgi,pano,spr,benzyn,benzy,klas,alufelg,enzyna,alufel,sce,zadb,ram,man,150,105,felgi,bdb,spo,90km,gwa,pcja,felg,tec,citroën,focus,astra,jak,picasso,megan,astr,krajowy,qashqai,qashqa,zarej,yaris,nav,opc,reje,iii,kombi,jest,pic,zar,tronik,mega,dam,con,krajow,dti,polski,bard,alu,pan,ara,tom,aut,rej,nie,str,hdi,kom,seria,seri,kraj,szyb,golf,przed,polecam,poleca,gol,ins,nowy,wszy,titanium,kóra,wers,wer,rav,naw,sta,fab,pier,oryginal,pol,peł,okazja,okaz,orygin,skó,tra,oryg,prz,dach,wyp,super,grand,sup,lift,110,000,turb,bezw,avant,rok,lif,diesel,diese,idea,wła,tyś,pas,tdci,tys,cic,ser,dci,100,tdc,tour,bieg,tempo,140,mały,pryw,zam,cam,osob,ory,320,oso,tel,park,mondeo,monde,crd,nic,xen,avensis,bog,mat,40km,ben,kod,oro,gran,120,130,temp,sel,tdi,par,pro,fote,tem,sat,fot,tani,170,grza,pdc,octavia,octavi,sto,max,cta,pod,szy,ele,van,ass,grz,nisk,bez,kier,ani,and,engine_type_Benzyna,engine_type_Diesel,engine_type_Hybryda,engine_type_Elektryczny,province_slaskie,province_mazowieckie,province_lódzkie,province_lubelskie,province_malopolskie,province_wielkopolskie,province_warmińsko-mazurskie,province_lubuskie,province_kujawsko-pomorskie,province_podlaskie,province_podkarpackie,province_pomorskie,province_opolskie,province_swietokrzyskie,province_region midtjylland,province_zylina,province_moravian-silesian region,province_noord-brabant,province_wilno,province_noord-holland,province_region hovestaden,province_nordrhein-westfalen,province_languedoc-roussillon-midi-pyrénées,province_hessen,province_berlin,province_niedersachsen
0,1,Mazda CX-3,69700,150KM 6AT 4x4 SkyPASSION (+ Biala skóra + Safe...,51015.0,2015,1998,,Katowice,,NaN,cx 3 1 6at skysion biala s safety i,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Peugeot Partner,19500,Opłacony~Dokumentacja Przebiegu,180000.0,2009,1600,,Garbatka-Letnisko,,Do negocjacji,tner ay dokumentacja eu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Kia Sportage,26800,Opłacona~Napęd 4x4,165000.0,2009,2000,,Garbatka-Letnisko,,Do negocjacji,age aa napęd,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [75]:
X = pd.concat([x.data.iloc[: , [4,5,6]], x.data.iloc[: , 12:319]], axis = 1, sort = False)

In [76]:
X.to_csv('X_21_06_2020.csv')

In [77]:
stand_scale = StandardScaler()

In [78]:
X_scaled = stand_scale.fit_transform(X)

In [82]:
lin_reg = LogisticRegression()

In [ ]:
lin_reg.fit(X_scaled, y)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [ ]:
y_pred = lin_reg.predict(y)

In [22]:
X.data.add_dummies()

,mileage_km
0,51015.0
1,180000.0
2,165000.0
3,238000.0
4,136000.0
...,...
4827,107000.0
4828,135340.0
4829,157500.0
4830,206000.0


In [14]:
x.add_dummies2(variables)

TypeError: first argument must be string or compiled pattern

In [4]:
results = pd.read_csv('results.csv')

In [55]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [58]:
data2 = AddDummy(data, 'title', 'ford')

In [3]:
x.data

NameError: name 'x' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [20]:
pipe = make_pipeline(StandardScaler(), LogisticRegression(priors=None))

['Ala', 'MA', 'KOtA']